In [1]:
import open3d as o3d
import numpy as np

# Load the STL files for stump and socket
stump_mesh = o3d.io.read_triangle_mesh("stump.stl")
socket_mesh = o3d.io.read_triangle_mesh("socket.stl")

# Convert the meshes to point clouds for easier handling if necessary
stump_pcd = stump_mesh.sample_points_uniformly(number_of_points=2000)
socket_pcd = socket_mesh.sample_points_uniformly(number_of_points=2000)

# Visualize the initial stump and socket
o3d.visualization.draw_geometries([stump_mesh, socket_mesh], window_name="Initial Stump and Socket")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
import copy

# Compute centroids
stump_center = stump_pcd.get_center()
socket_center = socket_pcd.get_center()

# Create deep copies of the original meshes for centered versions
centered_stump_mesh = copy.deepcopy(stump_mesh)
centered_socket_mesh = copy.deepcopy(socket_mesh)

# Create deep copies of the original meshes for centered versions
centered_stump_pcd = copy.deepcopy(stump_pcd)
centered_socket_pcd = copy.deepcopy(socket_pcd)

# Translate the copied meshes to center them
centered_stump_mesh.translate(-stump_center)
centered_socket_mesh.translate(-socket_center)

centered_stump_pcd.translate(-stump_center)
centered_socket_pcd.translate(-socket_center)

o3d.visualization.draw_geometries([centered_stump_pcd, centered_socket_pcd], window_name="Centered Stump and Socket")


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [9]:
# Find the Z-coordinate of the stump tip (lowest point in Z)
socket_bottom_z = np.min(np.asarray(centered_socket_pcd.points)[:, 2])
stump_bottom_z = np.min(np.asarray(centered_stump_pcd.points)[:, 2])
stump_top_z = np.max(np.asarray(centered_stump_pcd.points)[:, 2])
print (socket_bottom_z, stump_bottom_z, stump_top_z)

# Calculate required translation along the Z-axis
desired_depth = -70  # some amount of cm into the socket, can be changed 
current_difference_z = socket_bottom_z - stump_top_z
stump_translation_z = desired_depth-current_difference_z 
translated_and_centered_stump_pcd = copy.deepcopy(centered_stump_pcd)
translated_and_centered_stump_mesh = copy.deepcopy(centered_stump_mesh)

# Translate the stump along Z-axis
translated_and_centered_stump_pcd.translate([0, 0, -stump_translation_z])
translated_and_centered_stump_mesh.translate([0, 0, -stump_translation_z])  # Translate original mesh as well

# Visualize the positioned stump within the socket
o3d.visualization.draw_geometries([translated_and_centered_stump_pcd, centered_socket_pcd], window_name="Positioned Stump in Socket")
# o3d.visualization.draw_geometries([translated_and_centered_stump_pcd, centered_stump_pcd], window_name="Positioned Stump in Socket")
# o3d.visualization.draw_geometries([translated_and_centered_stump_mesh, centered_socket_mesh], window_name="Positioned Stump in Socket")


-112.39074843111914 -30.794975756232247 84.96925908073675


In [ ]:
electrode_positions = [
    [0,0,0]
    ] #these are coordinates provided by william
# Calculate center for scaling
# Here, we must calculate the maximum diameter, and scale until we scale by a certain factor
scaling_factor = 1.5 # Adjust to control gap size
scaled_vertices = []

scaled_stump_mesh = copy.deepcopy(translated_and_centered_stump_mesh)
stump_new_center = translated_and_centered_stump_pcd.get_center()

# Convert stump vertices to NumPy array for manipulation
stump_vertices = copy.deepcopy(np.asarray(scaled_stump_mesh.vertices))
np.append(stump_vertices, electrode_positions)
# Scale non-electrode points outward from the center
for point in stump_vertices:
    if any(np.allclose(point, electrode, atol=1e-3) for electrode in electrode_positions):
        scaled_vertices.append(point)  # Keep electrode points fixed
    else:
        direction = point - stump_new_center
        scaled_point = stump_new_center + direction * scaling_factor #Considers center to be 0,0,Z but check this
        scaled_vertices.append(scaled_point)

# Update stump mesh with scaled vertices for inner socket surface
scaled_stump_mesh.vertices = o3d.utility.Vector3dVector(scaled_vertices)

stump_bottom_z = np.min(np.asarray(translated_and_centered_stump_pcd.points)[:, 2])
stump_top_z = np.max(np.asarray(centered_stump_pcd.points)[:, 2])

scaled_stump_pcd = scaled_stump_mesh.sample_points_uniformly(number_of_points=2000)

# Visualize the scaled stump
o3d.visualization.draw_geometries([scaled_stump_pcd, translated_and_centered_stump_mesh], window_name="Scaled Stump for Inner Socket Surface")


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Add electrode positions

In [19]:
from sklearn.neighbors import NearestNeighbors

# Function to calculate normals and translate electrodes along the normal direction
def calculate_normals(mesh, points, neighbors=10):
    normals = []
    mesh_points = np.asarray(mesh.vertices)
    nbrs = NearestNeighbors(n_neighbors=neighbors).fit(mesh_points)
    for point in points:
        _, indices = nbrs.kneighbors([point])
        # Compute normal as average cross-product of nearest neighbors
        v1 = mesh_points[indices[0][1]] - point
        v2 = mesh_points[indices[0][2]] - point
        normal = np.cross(v1, v2)
        normal /= np.linalg.norm(normal)  # Normalize the vector
        normals.append(normal)
    return np.array(normals)

# Calculate normals at each electrode and translate outward
normals = calculate_normals(stump_mesh, electrode_positions)
offset_distance = 0.01  # Adjust distance
translated_electrodes = electrode_positions + normals * offset_distance

# Visualize moved electrodes
electrode_pcd = o3d.geometry.PointCloud()
electrode_pcd.points = o3d.utility.Vector3dVector(translated_electrodes)
o3d.visualization.draw_geometries([stump_mesh, socket_mesh, electrode_pcd], window_name="Electrode Positions Adjusted")



KeyboardInterrupt



In [ ]:
# Visualize cylindrical holes for electrodes
holes = []
hole_radius = 0.005  # Adjust hole size
for electrode in translated_electrodes:
    # Create a cylinder for each electrode
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=hole_radius, height=0.02)
    cylinder.paint_uniform_color([1, 0, 0])  # Color for visualization
    cylinder.translate(electrode)  # Position at electrode
    holes.append(cylinder)

# Visualize the final setup
o3d.visualization.draw_geometries([socket_mesh, stump_mesh, electrode_pcd] + holes, window_name="Final Socket with Electrode Positions")


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Perform boolean subtraction